In [1]:
from mmcv import Config
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (
    build_dataloader,
    build_dataset,
    replace_ImageToTensor)

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [2]:
classes = (
    "UNKNOWN", "General trash", "Paper", 
    "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", 
    "Battery", "Clothing")

#get config file
#cfg = Config.fromfile('/opt/ml/code/Swin-Transformer-Object-Detection-master/configs/swin/mask_rcnn_swin_small_patch4_window7_mstrain_480-800_adamw_3x_coco.py')
cfg = Config.fromfile('/opt/ml/code/Swin-Transformer-Object-Detection-master/configs/swin/cascade_mask_rcnn_swin_base.py')
#cfg = Config.fromfile('/opt/ml/code/Swin-Transformer-Object-Detection-master/configs/swin/htc_swin_small.py')
#cfg = Config.fromfile('/opt/ml/code/Swin-Transformer-Object-Detection-master/configs/swin/cascade_mask_rcnn_swin_small_patch4_window7_mstrain_480-800_giou_4conv1f_adamw_3x_coco.py')

#change dataset config
PREFIX = '/opt/ml/input/data/'
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'
cfg.data.train.seg_prefix=PREFIX  
#cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.train.seg_prefix=PREFIX  
#cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.train.seg_prefix=PREFIX  
#cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.runner.max_epochs = 16

#cfg.model.roi_head.bbox_head.num_classes = 11

cfg.seed=42
cfg.data.samples_per_gpu = 2 #batch size
#cfg.data.workers_per_gpu = 1
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/htc swin small'

#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.optimizer_config.use_fp16 = None

In [3]:
#group_name = 'efficientnet nas fpn'; 
project_name = 'pstage3_det'; run_name = 'base with small4'
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['entity'] = project_name
#cfg.log_config.hooks[1].init_kwargs['group']=group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

In [4]:
model = build_detector(cfg.model)

2021-05-20 08:03:06,881 - mmdet - INFO - load model from: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade pafpn base with small/swin_base_patch4_window7_224_22k (1).pth
2021-05-20 08:03:08,162 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [5]:
from mmcv.runner import load_checkpoint
#checkpoint = load_checkpoint(model, "/opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/mask rcnn swin small/mask_rcnn_swin_small_patch4_window7.pth", map_location='cuda:0')
#checkpoint = load_checkpoint(model, "/opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade mask rcnn swin base/cascade_mask_rcnn_swin_base_patch4_window7.pth", map_location='cuda:0')
checkpoint = load_checkpoint(model, "/opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/htc swin small/epoch_11.pth", map_location='cuda:0')

Use load_from_local loader


In [6]:
dataset = build_dataset(cfg.data.train)

loading annotations into memory...
Done (t=4.24s)
creating index...
index created!
/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2611: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [7]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-20 08:03:18,719 - mmdet - INFO - Start running, host: root@4a5a931d58fa, work_dir: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/htc swin small
2021-05-20 08:03:18,719 - mmdet - INFO - workflow: [('train', 1)], max: 16 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=0.81s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-20 08:05:01,295 - mmdet - INFO - Epoch [1][60/1636]	lr: 1.000e-05, eta: 11:39:58, time: 1.608, data_time: 0.069, memory: 18426, loss_rpn_cls: 0.0432, loss_rpn_bbox: 0.0299, s0.loss_cls: 0.2202, s0.acc: 92.8418, s0.loss_bbox: 0.2334, s0.loss_mask: 0.2252, s1.loss_cls: 0.1073, s1.acc: 92.9644, s1.loss_bbox: 0.0963, s1.loss_mask: 0.1039, s2.loss_cls: 0.0519, s2.acc: 93.0177, s2.loss_bbox: 0.0370, s2.loss_mask: 0.0476, loss: 1.1958
2021-05-20 08:06:34,866 - mmdet - INFO - Epoch [1][120/1636]	lr: 1.000e-05, eta: 11:27:48, time: 1.560, data_time: 0.026, memory: 18429, loss_rpn_cls: 0.0411, loss_rpn_bbox: 0.0303, s0.loss_cls: 0.2384, s0.acc: 92.5570, s0.loss_bbox: 0.2662, s0.loss_mask: 0.2200, s1.loss_cls: 0.1158, s1.acc: 92.8680, s1.loss_bbox: 0.1063, s1.loss_mask: 0.0975, s2.loss_cls: 0.0551, s2.acc: 93.0713, s2.loss_bbox: 0.0391, s2.loss_mask: 0.0440, loss: 1.2539
2021-05-20 08:08:08,190 - mmdet - INFO - Epoch [1][180/1636]	lr: 1.000e-05, eta: 11:22:06, time: 1.555, data_time: 0.02

KeyboardInterrupt: 

In [7]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 16:20:07,251 - mmdet - INFO - Start running, host: root@4a5a931d58fa, work_dir: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade pafpn base with small
2021-05-19 16:20:07,252 - mmdet - INFO - workflow: [('train', 1)], max: 16 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.13s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-19 16:20:51,980 - mmdet - INFO - Epoch [1][20/655]	lr: 3.896e-07, eta: 5:37:32, time: 1.936, data_time: 0.151, memory: 16577, loss_rpn_cls: 0.0823, loss_rpn_bbox: 0.0400, s0.loss_cls: 0.2556, s0.acc: 92.0410, s0.loss_bbox: 0.2832, s0.loss_mask: 0.2584, s1.loss_cls: 0.1214, s1.acc: 92.4578, s1.loss_bbox: 0.1169, s1.loss_mask: 0.1211, s2.loss_cls: 0.0597, s2.acc: 92.7271, s2.loss_bbox: 0.0437, s2.loss_mask: 0.0559, loss: 1.4384
2021-05-19 16:21:27,025 - mmdet - INFO - Epoch [1][40/655]	lr: 7.892e-07, eta: 5:20:53, time: 1.752, data_time: 0.023, memory: 16577, loss_rpn_cls: 0.0578, loss_rpn_bbox: 0.0440, s0.loss_cls: 0.2260, s0.acc: 92.5244, s0.loss_bbox: 0.2610, s0.loss_mask: 0.2698, s1.loss_cls: 0.1109, s1.acc: 92.5018, s1.loss_bbox: 0.1066, s1.loss_mask: 0.1186, s2.loss_cls: 0.0523, s2.acc: 93.3482, s2.loss_bbox: 0.0400, s2.loss_mask: 0.0524, loss: 1.3395
2021-05-19 16:22:02,830 - mmdet - INFO - Epoch [1][60/655]	lr: 1.189e-06, eta: 5:17:09, time: 1.790, data_time: 0.021, memor

KeyboardInterrupt: 

In [7]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 12:20:47,120 - mmdet - INFO - Start running, host: root@4a5a931d58fa, work_dir: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade pafpn base with small
2021-05-19 12:20:47,121 - mmdet - INFO - workflow: [('train', 1)], max: 16 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.16s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-19 12:21:26,677 - mmdet - INFO - Epoch [1][20/655]	lr: 3.896e-06, eta: 4:46:45, time: 1.645, data_time: 0.147, memory: 13502, loss_rpn_cls: 1.6085, loss_rpn_bbox: 0.1444, s0.loss_cls: 2.4901, s0.acc: 14.5142, s0.loss_bbox: 0.1906, s0.loss_mask: 1.3284, s1.loss_cls: 1.0906, s1.acc: 18.1348, s1.loss_bbox: 0.0257, s1.loss_mask: 0.7880, s2.loss_cls: 0.6423, s2.acc: 0.7593, s2.loss_bbox: 0.0027, s2.loss_mask: 0.2998, loss: 8.6110
2021-05-19 12:21:57,518 - mmdet - INFO - Epoch [1][40/655]	lr: 7.892e-06, eta: 4:37:15, time: 1.542, data_time: 0.021, memory: 13566, loss_rpn_cls: 0.6408, loss_rpn_bbox: 0.1198, s0.loss_cls: 2.3447, s0.acc: 21.2036, s0.loss_bbox: 0.2265, s0.loss_mask: 1.0999, s1.loss_cls: 1.0182, s1.acc: 51.9312, s1.loss_bbox: 0.0301, s1.loss_mask: 0.6986, s2.loss_cls: 0.6009, s2.acc: 6.7310, s2.loss_bbox: 0.0034, s2.loss_mask: 0.2549, loss: 7.0378
2021-05-19 12:22:28,993 - mmdet - INFO - Epoch [1][60/655]	lr: 1.189e-05, eta: 4:35:35, time: 1.574, data_time: 0.025, memory:

KeyboardInterrupt: 

In [7]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-19 08:53:09,164 - mmdet - INFO - Start running, host: root@4a5a931d58fa, work_dir: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade pafpn base with small
2021-05-19 08:53:09,164 - mmdet - INFO - workflow: [('train', 1)], max: 16 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.20s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-19 08:53:53,159 - mmdet - INFO - Epoch [1][20/655]	lr: 3.896e-07, eta: 5:30:47, time: 1.897, data_time: 0.149, memory: 15653, loss_rpn_cls: 0.0729, loss_rpn_bbox: 0.0512, s0.loss_cls: 0.2636, s0.acc: 91.5796, s0.loss_bbox: 0.3585, s0.loss_mask: 0.4488, s1.loss_cls: 0.1273, s1.acc: 91.8730, s1.loss_bbox: 0.1484, s1.loss_mask: 0.2345, s2.loss_cls: 0.0595, s2.acc: 92.2678, s2.loss_bbox: 0.0501, s2.loss_mask: 0.1165, loss: 1.9312
2021-05-19 08:54:28,112 - mmdet - INFO - Epoch [1][40/655]	lr: 7.892e-07, eta: 5:17:07, time: 1.748, data_time: 0.024, memory: 16534, loss_rpn_cls: 0.0756, loss_rpn_bbox: 0.0414, s0.loss_cls: 0.2677, s0.acc: 91.6064, s0.loss_bbox: 0.3294, s0.loss_mask: 0.4094, s1.loss_cls: 0.1299, s1.acc: 91.8587, s1.loss_bbox: 0.1372, s1.loss_mask: 0.2134, s2.loss_cls: 0.0596, s2.acc: 92.4342, s2.loss_bbox: 0.0474, s2.loss_mask: 0.1040, loss: 1.8150
2021-05-19 08:55:03,615 - mmdet - INFO - Epoch [1][60/655]	lr: 1.189e-06, eta: 5:13:46, time: 1.775, data_time: 0.020, memor

KeyboardInterrupt: 

In [7]:
train_detector(model, dataset, cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-18 05:14:52,479 - mmdet - INFO - Start running, host: root@4a5a931d58fa, work_dir: /opt/ml/code/Swin-Transformer-Object-Detection-master/work_dirs/cascade pafpn base with small
2021-05-18 05:14:52,480 - mmdet - INFO - workflow: [('train', 1)], max: 16 epochs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Done (t=1.14s)
creating index...
index created!
wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-18 05:15:37,015 - mmdet - INFO - Epoch [1][20/655]	lr: 3.896e-08, eta: 5:35:29, time: 1.924, data_time: 0.146, memory: 16673, loss_rpn_cls: 0.0430, loss_rpn_bbox: 0.0326, s0.loss_cls: 0.1965, s0.acc: 93.5229, s0.loss_bbox: 0.2588, s0.loss_mask: 0.2051, s1.loss_cls: 0.0894, s1.acc: 94.2165, s1.loss_bbox: 0.1080, s1.loss_mask: 0.0913, s2.loss_cls: 0.0432, s2.acc: 94.4775, s2.loss_bbox: 0.0431, s2.loss_mask: 0.0411, loss: 1.1521
2021-05-18 05:16:12,249 - mmdet - INFO - Epoch [1][40/655]	lr: 7.892e-08, eta: 5:20:42, time: 1.762, data_time: 0.022, memory: 16673, loss_rpn_cls: 0.0435, loss_rpn_bbox: 0.0382, s0.loss_cls: 0.1702, s0.acc: 94.2529, s0.loss_bbox: 0.2370, s0.loss_mask: 0.2098, s1.loss_cls: 0.0796, s1.acc: 94.8359, s1.loss_bbox: 0.0963, s1.loss_mask: 0.0927, s2.loss_cls: 0.0393, s2.acc: 94.8179, s2.loss_bbox: 0.0375, s2.loss_mask: 0.0420, loss: 1.0862
2021-05-18 05:16:47,337 - mmdet - INFO - Epoch [1][60/655]	lr: 1.189e-07, eta: 5:14:56, time: 1.754, data_time: 0.018, memor

KeyboardInterrupt: 